In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

users = pd.read_table('ml-1m/users.dat',  sep = '::', header = None, names= ['ID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
ratings = pd.read_table('ml-1m/ratings.dat',  sep = '::', header = None, names= ['UserID', 'MovieID', 'Rating', 'Timestamp'])
movies = pd.read_table('ml-1m/movies.dat',  sep = '::', engine='python', encoding = "ISO-8859-1", header = None, names= ['ID', 'Title', 'Genre'])

/tmp/ipykernel_15433/1386717902.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_table('ml-1m/users.dat',  sep = '::', header = None, names= ['ID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
/tmp/ipykernel_15433/1386717902.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table('ml-1m/ratings.dat',  sep = '::', header = None, names= ['UserID', 'MovieID', 'Rating', 'Timestamp'])


In [ ]:
def compute_RMSE(U,V, M):
    U_V = np.dot(U,V)
    diff = np.subtract(M, U_V)
    RMSE = np.sqrt(np.nansum(np.square(diff))/np.count_nonzero(~np.isnan(M)))
    print(RMSE, "RMSE")
    return RMSE

def UV_dec(U, V, M, d):
    #U_r_s
    for r in range(U.shape[0]):
        for s in range(U.shape[1]):
            m = M[r,~np.isnan(M[r,:])]
            if(len(m)==0):
                continue
            idx = np.argwhere(~np.isnan(M[r,:])).reshape(-1)
            v = V[s, idx]
            p = np.dot(U[r,:],V[:, idx]) - (U[r,s] * V[s, idx])
            U[r,s] = np.sum(v*(m-p))/np.sum(np.square(v))
    #V_r_s
    for r in range(V.shape[0]):
        for s in range(V.shape[1]):
            m = M[~np.isnan(M[:,s]),s]
            if(len(m)==0):
                continue
            idx = np.argwhere(~np.isnan(M[:,s])).reshape(-1)
            u = U[idx, r]
            p = np.dot(U[idx,:],V[:,s]) - (U[idx,r] * V[r, s])
            V[r,s] = np.sum(u*(m-p))/np.sum(np.square(u))
    return U,V



In [ ]:
kf = KFold(n_splits = 5, random_state=42, shuffle = True)
d = 2
#print(ratings)
for trainid, testid in kf.split(ratings):
    #data_train = ratings.iloc[trainid]
    #data_test = ratings.iloc[testid]
    #for i in testid:
        #print(i)
    train = ratings.copy()
    train.loc[testid, "Rating"] = np.nan
    u_m = train.pivot(index = "UserID", columns = "MovieID", values = "Rating")
    M = u_m.to_numpy()
    a = u_m.stack().mean(skipna = True)
    columns = list(u_m.columns)
    
    test = ratings.copy()
    test.loc[trainid, "Rating"] = np.nan
    u_m_test = test.pivot(index = "UserID", columns = "MovieID", values = "Rating")
    M_test = u_m_test.to_numpy()

    #print(M)
    #print(M_test)

    U = np.full([u_m.shape[0],d], np.sqrt(a/d))
    V = np.full([d,u_m.shape[1]], np.sqrt(a/d))
    compute_RMSE(U,V, M)
    for i in range(5):
        U, V = UV_dec(U,V,M,d)
        compute_RMSE(U,V,M)
    
    test_score = compute_RMSE(U,V, M_test)
    print(test_score, "test RMSE")

In [ ]:
def compute_RMSE(U,M, rat_m):
    U_M = np.dot(U,M)
    diff = np.subtract(rat_m, U_M)
    RMSE = np.sqrt(np.nansum(np.square(diff))/np.count_nonzero(~np.isnan(rat_m)))
    print(RMSE, "RMSE")
    return RMSE

kf = KFold(n_splits = 5, random_state=42, shuffle = True)
    
eta = 0.005
reg = 0.05

num_factors = 10
idx = list(range(num_factors))
print(idx)

for trainid, testid in kf.split(ratings):
    train = ratings.copy()
    train.loc[testid, "Rating"] = np.nan
    u_m = train.pivot(index = "UserID", columns = "MovieID", values = "Rating")
    X = u_m.to_numpy()
    columns = list(u_m.columns)

    test = ratings.copy()
    test.loc[trainid, "Rating"] = np.nan
    u_m_test = test.pivot(index = "UserID", columns = "MovieID", values = "Rating")
    X_test = u_m_test.to_numpy()

    U = np.random.uniform(low = -0.01, high = 0.01, size=(u_m.shape[0], num_factors))
    M = np.random.uniform(low = -0.01, high = 0.01, size=(num_factors, u_m.shape[1]))
    error(U,M, X)
    for run in range(10):
        print(run)
        for i, j in np.ndindex(X.shape):
            if(np.isnan(X[i,j])):
                continue
            else:
                pred_x =np.dot(U[i,idx], M[idx, j])
                er = X[i,j] - pred_x
                U[i,idx] = [U[i,k] + eta * (2*er * M[k,j] - reg * U[i,k]) for k in range(num_factors)]
                M[idx, j] = [M[k,j] + eta * (2*er * U[i,k] - reg *M[k,j])  for k in range(num_factors)]
        error(U,M, rat_m)
    
    test_score = compute_RMSE(U,M, X_test)
    print(test_score, "test RMSE")
        
    
    




[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
3.7513325338605803 RMSE
0
2.7056860166626397 RMSE
1
1.157776406635839 RMSE
2
0.9780813339671093 RMSE
3
0.943693600676001 RMSE
4
0.9253061957532511 RMSE
5
0.9111596684023681 RMSE
6
0.899171143026507 RMSE
7


0.9111696284830575 RMSE


In [ ]:
movies.rename(columns = {'ID':'MovieID'}, inplace = True)
merged = pd.merge(ratings, movies, on = 'MovieID')
utility_matrix = merged.pivot(index = "UserID", columns = "MovieID", values = "Rating")
#print(utility_matrix)
#print(utility_matrix.iloc[0,:].dropna())

In [ ]:
d = 6

M = ratings.pivot(index = "UserID", columns = "MovieID", values = "Rating")
M = utility_matrix.to_numpy()
a = utility_matrix.stack().mean(skipna = True)

U = np.full([utility_matrix.shape[0],d], np.sqrt(a/d))
V = np.full([d,utility_matrix.shape[1]], np.sqrt(a/d))
print(len(np.dot(U,V)))
compute_RMSE(U,V, M)
for i in range(100):
    U, V = UV_dec(U,V,M,d)
    compute_RMSE(U,V,M)

In [ ]:
def compute_RMSE(U,V, M):
    U_V = np.dot(U,V)
    diff = np.subtract(M, U_V)
    RMSE = np.sqrt(np.nansum(np.square(diff))/np.count_nonzero(~np.isnan(M)))
    print(RMSE, "RMSE")
    return RMSE
def UV_dec(U, V, M, d):
    #U_r_s
    for r in range(U.shape[0]):
        for s in range(U.shape[1]):
            m = M[r,~np.isnan(M[r,:])]
            if(len(m)==0):
                continue
            idx = np.argwhere(~np.isnan(M[r,:])).reshape(-1)
            v = V[s, idx]
            p = np.dot(U[r,:],V[:, idx]) - (U[r,s] * V[s, idx])
            U[r,s] = np.sum(v*(m-p))/np.sum(np.square(v))
    #V_r_s
    for r in range(V.shape[0]):
        for s in range(V.shape[1]):
            m = M[~np.isnan(M[:,s]),s]
            if(len(m)==0):
                continue
            idx = np.argwhere(~np.isnan(M[:,s])).reshape(-1)
            u = U[idx, r]
            p = np.dot(U[idx,:],V[:,s]) - (U[idx,r] * V[r, s])
            V[r,s] = np.sum(u*(m-p))/np.sum(np.square(u))
def UV_dec(U, V, M, d):
    #U_r_s
    for r in range(U.shape[0]):
        for s in range(U.shape[1]):
            
            #flatten to 1-d array containing indexis nonblank entries
            idx = np.argwhere(~np.isnan(M[r,:])).reshape(-1)
            #for each nonblank entry do:
            nom, den = 0,0
            for j in idx:
                p = [0 if k==s else U[r,k] * V[k,j] for k in range(d)]
                nom += V[s,j] * (M[r,j] - np.sum(p))
                den += np.square(V[s,j])
            U[r,s] = nom/den
                
                
            
    
    #V_r_s
    for r in range(V.shape[0]):
        for s in range(V.shape[1]):
            idx = np.argwhere(~np.isnan(M[:,s])).reshape(-1)
            nom, den = 0,0
            for i in idx:
                p = [0 if k==r else U[i,k] * V[k,s] for k in range(d)]
                nom += U[i,r] * (M[i,s] - np.sum(p))
                den += np.square(U[i,r])
            V[r,s] = nom/den
    
    return U,V

In [ ]:
def UV_dec_2(U, V, M, d):
    k = [i for i in range(d)]
    #U_r_s
    for r in range(U.shape[0]):
        for s in range(U.shape[1]):
            m = M[r,~np.isnan(M[r,:])]
            idx = np.argwhere(~np.isnan(M[r,:])).reshape(-1)
            v = V[s, idx]
            p = np.dot(U[r,:],V[:, idx]) - (U[r,s] * V[s, idx])
            U[r,s] = np.sum(v*(m-p))/np.sum(np.square(v)) 
    
    #V_r_s
    for r in range(V.shape[0]):
        for s in range(V.shape[1]):
            idx = np.argwhere(~np.isnan(M[:,s])).reshape(-1)
            nom, den = 0,0
            for i in idx:
                p = [0 if k==r else U[i,k] * V[k,s] for k in range(d)]
                nom += U[i,r] * (M[i,s] - np.sum(p))
                den += np.square(U[i,r])
            V[r,s] = nom/den
    
    return U,V

In [ ]:
d = 2

utility_matrix = ratings.pivot(index = "UserID", columns = "MovieID", values = "Rating")
a = utility_matrix.stack().mean(skipna = True)

U = np.full([utility_matrix.shape[0],d], np.random.uniform(0,5))#np.sqrt(a/d))
V = np.full([d,utility_matrix.shape[1]], np.random.uniform(0,5))#np.sqrt(a/d))
compute_RMSE(U,V, M)
for i in range(100):
    U, V = UV_dec(U,V,M,d)
    compute_RMSE(U,V,M)
    
    
    

In [ ]:

        
    
U_V = np.dot(U,V)
#convert pandas to numpy 
M = utility_matrix.to_numpy()
print(M[0,~np.isnan(M[0,:])])
#print(len(M[0,~np.isnan(M[0,:])]))
#print(np.unique(M[0,~np.isnan(M[0,:])],return_counts = True))
#print(np.unique(ratings.loc[ratings["UserID"]==1, "Rating"], return_counts = True))
idx = np.argwhere(~np.isnan(M[0,:])).reshape(-1)
for i in idx:
    print(M[0,i])

RMSE = compute_RMSE(U_V, M)  



In [ ]:
incremental_update(U, V):
    
    